<a href="https://colab.research.google.com/github/raushan9jnv/Fine-tuning-Llama-3.2-3B-Instruct-To-Act-like-OpenAI-GPT-Models-On-FineTome-100k-Data-With_Unsloth/blob/main/Fine_tuning_Llama_3_2_3B_Instruct_To_Act_like_OpenAI_GPT_Models_On_FineTome_100k_Data_With_Unsloth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Behave Llama as GPT

In [ ]:
# gpu must be enabled else unsloth give error
pip install unsloth transformers trl

In [ ]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

In [ ]:
#this will downlaod 2.35 gb of model and other config jason, unsloth fast downloding enabled

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length= 2048,   #max sequence a model can process, context window size for this model, amount of token we can passing in the single turn of msg ex. in converstion, all within 2048 token
    load_in_4bit =True)    # reduce the memory with 4 bit, as it is 3 billion parameter model

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [ ]:
#parameter efficient fine tuning model
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # r=16 is rank parameter for Lora
    target_modules = ["q_proj", "k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"] # query  projection, key projection, value proj, output, gate, up, down...important
)

Unsloth 2025.3.19 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
tokenier = get_chat_template(tokenizer, chat_template= "llama-3.1")

In [ ]:
# it will download 117 MB of dataset from huggiface
#url of dataset: https://huggingface.co/datasets/mlabonne/FineTome-100k

dataset = load_dataset("mlabonne/FineTome-100k", split ="train")

In [ ]:
dataset = standardize_sharegpt(dataset)

In [ ]:
dataset

Dataset({
    features: ['conversations', 'source', 'score'],
    num_rows: 100000
})

In [ ]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [ ]:
dataset = dataset.map(
    lambda examples: {
        "text":[
            tokenier.apply_chat_template(convo, tokenize= False)
            for convo in examples["conversations"]
        ]
    },
    batched = True
)

In [ ]:
dataset[0]

{'conversations': [{'content': 'Explain what boolean operators are, what they do, and provide examples of how they can be used in programming. Additionally, describe the concept of operator precedence and provide examples of how it affects the evaluation of boolean expressions. Discuss the difference between short-circuit evaluation and normal evaluation in boolean expressions and demonstrate their usage in code. \n\nFurthermore, add the requirement that the code must be written in a language that does not support short-circuit evaluation natively, forcing the test taker to implement their own logic for short-circuit evaluation.\n\nFinally, delve into the concept of truthiness and falsiness in programming languages, explaining how it affects the evaluation of boolean expressions. Add the constraint that the test taker must write code that handles cases where truthiness and falsiness are implemented differently across different programming languages.',
   'role': 'user'},
  {'content': 

In [ ]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field ="text",
    max_seq_length = 2048,
    args= TrainingArguments(
        per_device_train_batch_size =2, #faster
        gradient_accumulation_steps = 4,
        warmup_steps=5,
        max_steps =60,
        learning_rate = 2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps =1,
        output_dir ="outputs"
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
# this will ask
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,000,000,000 (0.81% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: raushan9jnv to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.492200
2,2.629700
3,1.622000
4,1.940300
5,1.900700
6,1.977600
7,1.147200
8,1.859400
9,1.525300
10,1.452000


TrainOutput(global_step=60, training_loss=1.1827350278695425, metrics={'train_runtime': 634.539, 'train_samples_per_second': 0.756, 'train_steps_per_second': 0.095, 'total_flos': 6240291994460160.0, 'train_loss': 1.1827350278695425})

In [ ]:
model.save_pretrained("finetuned_model")

In [ ]:
inference_model,inference_tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./finetuned_model",
    max_seq_length = 2048,
    load_in_4bit = True
)

==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.50.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
text_prompts = [
    "what are the key principles of investment?"
]

for prompt in text_prompts:
  formatted_prompt = inference_tokenizer.apply_chat_template([{
      "role":"user",
      "content":prompt
  }],tokenize = False)

  model_inputs = inference_tokenizer(formatted_prompt,return_tensors="pt").to("cuda")
  generated_ids = inference_model.generate(
      **model_inputs,
      max_new_tokens= 512,
      temperature =0.7,
      do_sample=True,
      pad_token_id = inference_tokenizer.pad_token_id
  )
  response = inference_tokenizer.batch_decode(generated_ids, skip_special_tokens= True)[0]
  print(response)

system

Cutting Knowledge Date: December 2023
Today Date: 04 Apr 2025

user

what are the key principles of investment?assistant

1. **Risk Management**: Understanding and managing risk is a key principle of investment. This involves assessing the potential risks associated with an investment and taking steps to mitigate or manage them.
2. **Diversification**: Diversifying your portfolio by investing in different asset classes, sectors, or geographic regions can help reduce risk and increase potential returns.
3. **Long-term Focus**: Investing for the long-term is essential. Markets can be volatile, and short-term fluctuations can be significant. A long-term focus allows you to ride out market downturns and capture potential gains.
4. **Dollar-Cost Averaging**: Investing a fixed amount of money at regular intervals, regardless of market conditions, can help reduce the impact of market volatility.
5. **Tax Efficiency**: Understanding how taxes affect your investments is crucial. Conside

## Thanks